In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import auc
from sklearn import metrics

df = pd.read_csv('/Users/naresh/downloads/train.csv')
#df_train = df_train[0:46020]
#df_test = df_train[-30000:]
msk = np.random.rand(len(df)) < 0.7
df_train = df[msk]
df_test = df[~msk]
df_target = df_train['TARGET']
df_train = df_train.drop(['TARGET', 'ID'], axis=1)
df_test_target = df_test['TARGET']
df_test = df_test.drop(['TARGET','ID'], axis = 1)




probs = pd.DataFrame()
errors = pd.DataFrame()

X_train = df_train
y_train = df_target

X_train, X_test, y_train, y_test= train_test_split(X_train, y_train, test_size=0.3, random_state=0)
learning_rate = 0.01
n_estimators = 800
max_depth = 6
subsample = 0.9
colsample_bytree = 0.85
min_child_weight = 1  # default

eval_metrics = ['auc']
eval_sets = [(X_train, y_train), (X_test, y_test)]
xgb_model = XGBClassifier(seed=0, learning_rate=learning_rate, n_estimators=n_estimators,
                    min_child_weight=min_child_weight, max_depth=max_depth,
                    colsample_bytree=colsample_bytree, subsample=subsample,nthread = -1)
print("Fitting the model")
xgb = xgb_model.fit(X_train, y_train, eval_metric=eval_metrics, eval_set=eval_sets, verbose=False)
    
print("Predicting Probabilities")
probs['xgb'] = xgb.predict_proba(X_test)[:, -1]

print("Computing AUC")
auc_test = [xgb.evals_result_['validation_%d' % i]['auc'] for i in range(len(eval_sets))]
auc_test = np.array(auc_test, dtype=float).T

auc_best_round = np.argmax(auc_test, axis=0)
auc_best = [auc_test[auc_best_round[0], 0], auc_test[auc_best_round[1], 1]]

print('Best AUC train=%f (round=%d), test=%f (round=%d)' % (auc_best[0], auc_best_round[0], auc_best[1], auc_best_round[1]))
print('Validation')
test_probs = pd.DataFrame()
test_probs['xgb_valid'] = xgb.predict_proba(df_test)[:,-1]
print(test_probs['xgb_valid'].head())

fpr, tpr, thresholds = metrics.roc_curve(df_test_target, test_probs, pos_label=1)

a = float(auc(fpr,tpr))
print(a)

Fitting the model
Predicting Probabilities
Computing AUC
Best AUC train=0.913128 (round=799), test=0.839790 (round=744)
Validation
0    0.017874
1    0.012916
2    0.014823
3    0.161807
4    0.006301
Name: xgb_valid, dtype: float32
0.8284797407611536


In [46]:
ZERO_VARIANCE_COLUMNS = [
    'ind_var2_0', 'ind_var2', 'ind_var27_0', 'ind_var28_0', 'ind_var28', 'ind_var27', 'ind_var41', 'ind_var46_0',
    'ind_var46', 'num_var27_0', 'num_var28_0', 'num_var28', 'num_var27', 'num_var41', 'num_var46_0', 'num_var46',
    'saldo_var28', 'saldo_var27', 'saldo_var41', 'saldo_var46', 'imp_amort_var18_hace3', 'imp_amort_var34_hace3',
    'imp_reemb_var13_hace3', 'imp_reemb_var33_hace3', 'imp_trasp_var17_out_hace3', 'imp_trasp_var33_out_hace3',
    'num_var2_0_ult1', 'num_var2_ult1', 'num_reemb_var13_hace3', 'num_reemb_var33_hace3', 'num_trasp_var17_out_hace3',
    'num_trasp_var33_out_hace3', 'saldo_var2_ult1', 'saldo_medio_var13_medio_hace3'
]

CORRELATED_COLUMNS = [
    'ind_var29_0', 'ind_var29', 'num_var6', 'num_var29', 'ind_var13_medio', 'num_var13_medio_0', 'num_var13_medio',
    'num_meses_var13_medio_ult3', 'ind_var18', 'num_var18_0', 'num_var18', 'num_var20_0', 'num_var20', 'ind_var26',
    'ind_var25', 'ind_var32', 'ind_var34', 'ind_var37', 'ind_var39', 'num_var29_0', 'delta_imp_amort_var18_1y3',
    'num_var26', 'num_var25', 'num_var32', 'num_var34', 'delta_imp_amort_var34_1y3', 'num_var37', 'num_var39',
    'saldo_var29', 'saldo_medio_var13_medio_ult1', 'delta_num_aport_var13_1y3', 'delta_num_aport_var17_1y3',
    'delta_num_aport_var33_1y3', 'delta_num_reemb_var13_1y3', 'num_reemb_var13_ult1', 'delta_num_reemb_var17_1y3',
    'delta_num_reemb_var33_1y3', 'num_reemb_var33_ult1', 'delta_num_trasp_var17_in_1y3',
    'delta_num_trasp_var17_out_1y3', 'delta_num_trasp_var33_in_1y3', 'delta_num_trasp_var33_out_1y3',
    'num_trasp_var33_out_ult1', 'delta_num_venta_var44_1y3'
]

In [2]:
from sklearn.grid_search import GridSearchCV
from xgboost.sklearn import XGBClassifier

param_test1 = {
 'max_depth': [i for i in range(3,10,2)],
 'min_child_weight':[i for i in range(1,6,2)]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X_train,y_train)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

/Users/naresh/Downloads/scikit-learn-master/sklearn/grid_search.py:43: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


([mean: 0.83190, std: 0.00862, params: {'max_depth': 3, 'min_child_weight': 1},
  mean: 0.83239, std: 0.00846, params: {'max_depth': 3, 'min_child_weight': 3},
  mean: 0.83186, std: 0.00852, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: 0.82965, std: 0.00897, params: {'max_depth': 5, 'min_child_weight': 1},
  mean: 0.82928, std: 0.01052, params: {'max_depth': 5, 'min_child_weight': 3},
  mean: 0.82934, std: 0.00939, params: {'max_depth': 5, 'min_child_weight': 5},
  mean: 0.82435, std: 0.00783, params: {'max_depth': 7, 'min_child_weight': 1},
  mean: 0.82321, std: 0.00914, params: {'max_depth': 7, 'min_child_weight': 3},
  mean: 0.82461, std: 0.00924, params: {'max_depth': 7, 'min_child_weight': 5},
  mean: 0.81612, std: 0.00841, params: {'max_depth': 9, 'min_child_weight': 1},
  mean: 0.82074, std: 0.00813, params: {'max_depth': 9, 'min_child_weight': 3},
  mean: 0.82099, std: 0.01036, params: {'max_depth': 9, 'min_child_weight': 5}],
 {'max_depth': 3, 'min_child_weight': 3

In [49]:
from xgboost import plot_importance

#df_train = df_train.drop(['ind_var2_0', 'ind_var2', 'ind_var27_0', 'ind_var28_0', 'ind_var28', 'ind_var27', 'ind_var41', 'ind_var46_0',
    #'ind_var46', 'num_var27_0', 'num_var28_0', 'num_var28', 'num_var27', 'num_var41', 'num_var46_0', 'num_var46',
    #'saldo_var28', 'saldo_var27', 'saldo_var41', 'saldo_var46', 'imp_amort_var18_hace3', 'imp_amort_var34_hace3',
    #'imp_reemb_var13_hace3', 'imp_reemb_var33_hace3', 'imp_trasp_var17_out_hace3', 'imp_trasp_var33_out_hace3',
    #'num_var2_0_ult1', 'num_var2_ult1', 'num_reemb_var13_hace3', 'num_reemb_var33_hace3', 'num_trasp_var17_out_hace3',
    #'num_trasp_var33_out_hace3', 'saldo_var2_ult1', 'saldo_medio_var13_medio_hace3'],axis = 1)
#df_train = df_train.drop(['ind_var29_0', 'ind_var29', 'num_var6', 'num_var29', 'ind_var13_medio', 'num_var13_medio_0', 'num_var13_medio',
    #'num_meses_var13_medio_ult3', 'ind_var18', 'num_var18_0', 'num_var18', 'num_var20_0', 'num_var20', 'ind_var26',
    #'ind_var25', 'ind_var32', 'ind_var34', 'ind_var37', 'ind_var39', 'num_var29_0', 'delta_imp_amort_var18_1y3',
    #'num_var26', 'num_var25', 'num_var32', 'num_var34', 'delta_imp_amort_var34_1y3', 'num_var37', 'num_var39',
    #'saldo_var29', 'saldo_medio_var13_medio_ult1', 'delta_num_aport_var13_1y3', 'delta_num_aport_var17_1y3',
    #'delta_num_aport_var33_1y3', 'delta_num_reemb_var13_1y3', 'num_reemb_var13_ult1', 'delta_num_reemb_var17_1y3',
    #'delta_num_reemb_var33_1y3', 'num_reemb_var33_ult1', 'delta_num_trasp_var17_in_1y3',
    #'delta_num_trasp_var17_out_1y3', 'delta_num_trasp_var33_in_1y3', 'delta_num_trasp_var33_out_1y3',
    #'num_trasp_var33_out_ult1', 'delta_num_venta_var44_1y3'],axis = 1)
#df_test = df_test.drop(['ind_var29_0', 'ind_var29', 'num_var6', 'num_var29', 'ind_var13_medio', 'num_var13_medio_0', 'num_var13_medio',
    #'num_meses_var13_medio_ult3', 'ind_var18', 'num_var18_0', 'num_var18', 'num_var20_0', 'num_var20', 'ind_var26',
    #'ind_var25', 'ind_var32', 'ind_var34', 'ind_var37', 'ind_var39', 'num_var29_0', 'delta_imp_amort_var18_1y3',
    #'num_var26', 'num_var25', 'num_var32', 'num_var34', 'delta_imp_amort_var34_1y3', 'num_var37', 'num_var39',
    #'saldo_var29', 'saldo_medio_var13_medio_ult1', 'delta_num_aport_var13_1y3', 'delta_num_aport_var17_1y3',
    #'delta_num_aport_var33_1y3', 'delta_num_reemb_var13_1y3', 'num_reemb_var13_ult1', 'delta_num_reemb_var17_1y3',
    ##'delta_num_reemb_var33_1y3', 'num_reemb_var33_ult1', 'delta_num_trasp_var17_in_1y3',
    #'delta_num_trasp_var17_out_1y3', 'delta_num_trasp_var33_in_1y3', 'delta_num_trasp_var33_out_1y3',
    #'num_trasp_var33_out_ult1', 'delta_num_venta_var44_1y3'], axis = 1)
#df_test = df_test.drop(['ind_var2_0', 'ind_var2', 'ind_var27_0', 'ind_var28_0', 'ind_var28', 'ind_var27', 'ind_var41', 'ind_var46_0',
    #'ind_var46', 'num_var27_0', 'num_var28_0', 'num_var28', 'num_var27', 'num_var41', 'num_var46_0', 'num_var46',
    #'saldo_var28', 'saldo_var27', 'saldo_var41', 'saldo_var46', 'imp_amort_var18_hace3', 'imp_amort_var34_hace3',
    ##'imp_reemb_var13_hace3', 'imp_reemb_var33_hace3', 'imp_trasp_var17_out_hace3', 'imp_trasp_var33_out_hace3',
    #'num_var2_0_ult1', 'num_var2_ult1', 'num_reemb_var13_hace3', 'num_reemb_var33_hace3', 'num_trasp_var17_out_hace3',
    #'num_trasp_var33_out_hace3', 'saldo_var2_ult1', 'saldo_medio_var13_medio_hace3'],axis = 1)

#Run XGBoost model


X_train['num_zero'] = (X_train == 0).sum(axis=1)
X_test['num_zero'] = (X_test == 0).sum(axis=1)

print(len(X_test))

X_train, X_test, y_train, y_test= train_test_split(X_train, y_train, test_size=0.3, random_state=0)
learning_rate = 0.01
n_estimators = 800
max_depth = 3
subsample = 0.9
colsample_bytree = 0.85
min_child_weight = 3  # default

eval_metrics = ['auc']
eval_sets = [(X_train, y_train), (X_test, y_test)]
xgb_model = XGBClassifier(seed=0, learning_rate=learning_rate, n_estimators=n_estimators,
                    min_child_weight=min_child_weight, max_depth=max_depth,
                    colsample_bytree=colsample_bytree, subsample=subsample,nthread = -1)
print("Fitting the model")
xgb = xgb_model.fit(X_train, y_train, eval_metric=eval_metrics, eval_set=eval_sets, verbose=False)
    
print("Predicting Probabilities")
probs_new = pd.DataFrame()
probs_new['xgb'] = xgb_model.predict_proba(X_test)[:, -1]

print("Computing AUC")
auc_test = [xgb.evals_result_['validation_%d' % i]['auc'] for i in range(len(eval_sets))]
auc_test = np.array(auc_test, dtype=float).T

auc_best_round = np.argmax(auc_test, axis=0)
auc_best = [auc_test[auc_best_round[0], 0], auc_test[auc_best_round[1], 1]]

print('Best AUC train=%f (round=%d), test=%f (round=%d)' % (auc_best[0], auc_best_round[0], auc_best[1], auc_best_round[1]))
print('Validation')





importances = xgb.booster().get_fscore()
print(importances)



/Users/naresh/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/naresh/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


15977
Fitting the model
Predicting Probabilities
Computing AUC
Best AUC train=0.868257 (round=799), test=0.847360 (round=799)
Validation
{'num_op_var39_ult3': 2, 'ind_var12_0': 9, 'ind_var30_0': 6, 'saldo_medio_var8_ult3': 2, 'var38': 670, 'saldo_medio_var12_ult3': 11, 'num_op_var41_ult1': 9, 'num_op_var41_comer_ult3': 7, 'saldo_var42': 38, 'imp_op_var41_efect_ult1': 81, 'saldo_var5': 79, 'num_op_var39_efect_ult3': 2, 'imp_op_var41_comer_ult1': 30, 'saldo_medio_var5_ult1': 58, 'num_op_var41_hace2': 14, 'num_var22_ult1': 187, 'num_var43_recib_ult1': 3, 'ind_var43_recib_ult1': 1, 'num_var39_0': 1, 'var3': 90, 'saldo_medio_var8_hace2': 2, 'imp_op_var39_efect_ult1': 22, 'ind_var24_0': 1, 'num_meses_var5_ult3': 58, 'ind_var25_0': 2, 'num_op_var39_hace2': 6, 'num_med_var22_ult3': 21, 'saldo_medio_var12_ult1': 2, 'saldo_var37': 60, 'num_med_var45_ult3': 29, 'ind_var39_0': 1, 'num_var13': 1, 'imp_op_var39_comer_ult1': 14, 'num_var43_emit_ult1': 1, 'imp_trans_var37_ult1': 42, 'num_op_var39_come

In [56]:
from sklearn.neural_network import MLPClassifier

X_train = StandardScaler().fit_transform(X_train)

nn = MLPClassifier(hidden_layer_sizes=(200,50), max_iter=100, alpha=1e-4,
                    algorithm='adam', verbose=10, tol=1e-4, random_state=1,
                    learning_rate= 'adaptive')
X_test = StandardScaler().fit_transform(X_test)
df_test = StandardScaler().fit_transform(df_test)

nn.fit(X_train, y_train)
test_probs1 = pd.DataFrame()
test_probs1['mlp'] = nn.predict_proba(X_test)[:, -1]
print(len(df_test))
print(len(X_test))
print(test_probs1['mlp'].head())
print(len(test_probs1['mlp']))
fpr, tpr, thresholds = metrics.roc_curve(y_test, test_probs1, pos_label=1)


a = float(auc(fpr,tpr))
print(a)

Iteration 1, loss = 0.15667121
Iteration 2, loss = 0.13831011
Iteration 3, loss = 0.13465906
Iteration 4, loss = 0.13096057
Iteration 5, loss = 0.12976440
Iteration 6, loss = 0.12909735
Iteration 7, loss = 0.12704487
Iteration 8, loss = 0.12639400
Iteration 9, loss = 0.12465343
Iteration 10, loss = 0.12434712
Iteration 11, loss = 0.12430869
Iteration 12, loss = 0.12288689
Iteration 13, loss = 0.12077099
Iteration 14, loss = 0.11990932
Iteration 15, loss = 0.11955826
Iteration 16, loss = 0.11787604
Iteration 17, loss = 0.11787424
Iteration 18, loss = 0.11760271
Iteration 19, loss = 0.11634629
Iteration 20, loss = 0.11438073
Iteration 21, loss = 0.11408426
Iteration 22, loss = 0.11305997
Iteration 23, loss = 0.11261563
Iteration 24, loss = 0.11136189
Iteration 25, loss = 0.11126067
Iteration 26, loss = 0.11069287
Iteration 27, loss = 0.11169554
Iteration 28, loss = 0.11177548
Iteration 29, loss = 0.11076092
Training loss did not improve more than tol=0.000100 for two consecutive epochs. 

In [61]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, 0.2*test_probs1['mlp']+0.8*probs_new['xgb'])
a = float(auc(fpr,tpr))
print(a)

0.8431245477008673


In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

clf = AdaBoostClassifier(n_estimators=100)
clf.fit(X_train,y_train)

clf2 = RandomForestClassifier(n_estimators=100, max_depth=5, min_samples_split=1, random_state=0)
clf2.fit(X_train,y_train)

eclf = VotingClassifier(estimators=[('nn',nn ), ('xgbt', xgb_model),('ada',clf),('rf',clf2)], voting='soft', weights=[1,2,0.5,0.5])
eclf.fit(X_train,y_train)




Iteration 1, loss = 0.15667121
Iteration 2, loss = 0.13831011
Iteration 3, loss = 0.13465906
Iteration 4, loss = 0.13096057
Iteration 5, loss = 0.12976440
Iteration 6, loss = 0.12909735
Iteration 7, loss = 0.12704487
Iteration 8, loss = 0.12639400
Iteration 9, loss = 0.12465343
Iteration 10, loss = 0.12434712
Iteration 11, loss = 0.12430869
Iteration 12, loss = 0.12288689
Iteration 13, loss = 0.12077099
Iteration 14, loss = 0.11990932
Iteration 15, loss = 0.11955826
Iteration 16, loss = 0.11787604
Iteration 17, loss = 0.11787424
Iteration 18, loss = 0.11760271
Iteration 19, loss = 0.11634629
Iteration 20, loss = 0.11438073
Iteration 21, loss = 0.11408426
Iteration 22, loss = 0.11305997
Iteration 23, loss = 0.11261563
Iteration 24, loss = 0.11136189
Iteration 25, loss = 0.11126067
Iteration 26, loss = 0.11069287
Iteration 27, loss = 0.11169554
Iteration 28, loss = 0.11177548
Iteration 29, loss = 0.11076092
Training loss did not improve more than tol=0.000100 for two consecutive epochs. 

In [ ]:
test_probs2 = pd.DataFrame()
test_probs2['voting'] = eclf.predict_proba(df_test)[:, -1]
print(test_probs2['voting'].head())
print()
fpr, tpr, thresholds = metrics.roc_curve(df_test_target, test_probs2, pos_label=1)
a = float(auc(fpr,tpr))
print(a)


In [ ]:
from sklearn.grid_search import GridSearchCV
from xgboost.sklearn import XGBClassifier

param_test1 = {
 'max_depth': [i for i in range(3,12,1)],
 'colsample_bytree':np.linspace(0.8,0.9,1)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X_train,y_train)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [ ]:
df_final = pd.read_csv('/Users/naresh/downloads/sample_submission.csv')
df_tests = pd.read_csv('/Users/naresh/downloads/test.csv')



df_final['TARGET'] = test_probs2
print(df_final)
print(df_tests.head(5))